# 02. 音響効果の実装と可視化

様々な音響効果をプログラムで実装し、その効果を可視化します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, ifft, fftfreq
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 基本的な音響効果

In [ ]:
class AudioEffects:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def echo(self, signal, delay_ms=500, decay=0.6, num_echoes=3):
        """エコー効果"""
        delay_samples = int(delay_ms * self.sample_rate / 1000)
        
        # 出力信号の長さを計算
        output_length = len(signal) + delay_samples * num_echoes
        output = np.zeros(output_length)
        
        # 元の信号をコピー
        output[:len(signal)] = signal
        
        # エコーを追加
        for i in range(1, num_echoes + 1):
            start_idx = delay_samples * i
            end_idx = start_idx + len(signal)
            if end_idx <= len(output):
                output[start_idx:end_idx] += signal * (decay ** i)
                
        return output
        
    def reverb(self, signal, room_size=0.8, damping=0.5, wet_level=0.3):
        """リバーブ効果（簡易実装）"""
        # 複数の遅延ラインを使用したシンプルなリバーブ
        delays = [29, 37, 41, 43, 47, 53, 59, 61]  # プライム数の遅延
        
        output = signal.copy()
        
        for delay in delays:
            delay_samples = int(delay * room_size)
            if delay_samples > 0 and delay_samples < len(signal):
                # 遅延信号を作成
                delayed = np.zeros_like(signal)
                delayed[delay_samples:] = signal[:-delay_samples]
                
                # ダンピングを適用
                delayed *= damping
                
                # ミックス
                output += delayed * wet_level / len(delays)
                
        return output
        
    def chorus(self, signal, rate=1.5, depth=0.002, delay=0.02):
        """コーラス効果"""
        delay_samples = int(delay * self.sample_rate)
        max_depth_samples = int(depth * self.sample_rate)
        
        t = np.arange(len(signal)) / self.sample_rate
        
        # LFOによる遅延時間の変調
        lfo = np.sin(2 * np.pi * rate * t)
        variable_delay = delay_samples + max_depth_samples * lfo
        
        output = signal.copy()
        
        # 可変遅延の実装（線形補間）
        for i in range(len(signal)):
            delay_idx = i - variable_delay[i]
            
            if delay_idx >= 0 and delay_idx < len(signal) - 1:
                # 線形補間
                idx_floor = int(delay_idx)
                idx_ceil = idx_floor + 1
                frac = delay_idx - idx_floor
                
                interpolated = (signal[idx_floor] * (1 - frac) + 
                              signal[idx_ceil] * frac)
                output[i] += interpolated * 0.5
                
        return output
        
    def distortion(self, signal, gain=2.0, threshold=0.7):
        """歪み効果"""
        # ゲインを適用
        amplified = signal * gain
        
        # ソフトクリッピング
        output = np.zeros_like(amplified)
        
        # しきい値以下は線形
        linear_mask = np.abs(amplified) <= threshold
        output[linear_mask] = amplified[linear_mask]
        
        # しきい値以上はtanh関数でクリッピング
        clip_mask = ~linear_mask
        output[clip_mask] = np.sign(amplified[clip_mask]) * threshold * np.tanh(
            np.abs(amplified[clip_mask]) / threshold
        )
        
        return output
        
    def tremolo(self, signal, rate=6.0, depth=0.5):
        """トレモロ効果（振幅変調）"""
        t = np.arange(len(signal)) / self.sample_rate
        lfo = np.sin(2 * np.pi * rate * t)
        
        # 振幅変調
        modulation = 1 + depth * lfo
        return signal * modulation
        
    def vibrato(self, signal, rate=5.0, depth=0.01):
        """ビブラート効果（周波数変調）"""
        max_delay_samples = int(depth * self.sample_rate)
        
        t = np.arange(len(signal)) / self.sample_rate
        lfo = np.sin(2 * np.pi * rate * t)
        
        # 可変遅延による周波数変調の近似
        delay_variation = max_delay_samples * lfo
        
        output = np.zeros_like(signal)
        
        for i in range(len(signal)):
            delay_idx = i - delay_variation[i]
            
            if delay_idx >= 0 and delay_idx < len(signal) - 1:
                idx_floor = int(delay_idx)
                idx_ceil = idx_floor + 1
                frac = delay_idx - idx_floor
                
                output[i] = (signal[idx_floor] * (1 - frac) + 
                           signal[idx_ceil] * frac)
            elif delay_idx >= 0:
                output[i] = signal[int(delay_idx)]
                
        return output

# テスト信号の生成
sample_rate = 22050  # 軽量化のため
duration = 2.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# 楽器風の音（基音 + 倍音）
fundamental = 220  # A3
test_signal = (np.sin(2 * np.pi * fundamental * t) +
              0.5 * np.sin(2 * np.pi * fundamental * 2 * t) +
              0.3 * np.sin(2 * np.pi * fundamental * 3 * t))

# エンベロープを適用（音楽的に）
envelope = np.exp(-t * 0.5)  # 指数減衰
test_signal *= envelope

# 効果を適用
fx = AudioEffects(sample_rate)

effects = {
    '元の信号': test_signal,
    'エコー': fx.echo(test_signal, delay_ms=300, decay=0.5, num_echoes=3),
    'リバーブ': fx.reverb(test_signal, room_size=0.7, damping=0.6, wet_level=0.4),
    'コーラス': fx.chorus(test_signal, rate=1.2, depth=0.001, delay=0.015),
    '歪み': fx.distortion(test_signal, gain=3.0, threshold=0.5),
    'トレモロ': fx.tremolo(test_signal, rate=5.0, depth=0.7)
}

# 可視化
plt.figure(figsize=(18, 12))

for i, (effect_name, affected_signal) in enumerate(effects.items()):
    # 時間領域
    plt.subplot(3, 4, i*2 + 1)
    
    # 表示用に信号をトリミング
    display_signal = affected_signal[:len(t)]
    plt.plot(t[:4000], display_signal[:4000], linewidth=1)
    plt.title(f'{effect_name}（時間波形）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # 周波数領域
    plt.subplot(3, 4, i*2 + 2)
    
    # FFT
    fft_result = fft(display_signal)
    frequencies = fftfreq(len(display_signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    
    magnitude = np.abs(fft_result[positive_mask])
    plt.plot(frequencies[positive_mask], 20 * np.log10(magnitude + 1e-10), linewidth=1)
    plt.title(f'{effect_name}（スペクトル）')
    plt.ylabel('振幅 (dB)')
    plt.grid(True)
    plt.xlim(0, 2000)
    
    if i >= len(effects) - 2:  # 最後の2行
        plt.subplot(3, 4, i*2 + 1)
        plt.xlabel('時間 (秒)')
        plt.subplot(3, 4, i*2 + 2)
        plt.xlabel('周波数 (Hz)')

plt.tight_layout()
plt.show()

print("実装した音響効果:")
print("1. エコー: 遅延と減衰を伴う反復")
print("2. リバーブ: 空間の残響をシミュレート")
print("3. コーラス: 微細な遅延変調による音の厚み")
print("4. 歪み: 非線形増幅による音色変化")
print("5. トレモロ: 振幅の周期的変化")
print("6. ビブラート: 周波数の周期的変化")

## 2. フィルター効果

In [ ]:
class FilterEffects:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def lowpass_filter(self, signal, cutoff_freq, order=4):
        """ローパスフィルタ"""
        nyquist = self.sample_rate / 2
        normalized_cutoff = cutoff_freq / nyquist
        b, a = signal.butter(order, normalized_cutoff, btype='low')
        return signal.filtfilt(b, a, signal)
        
    def highpass_filter(self, signal, cutoff_freq, order=4):
        """ハイパスフィルタ"""
        nyquist = self.sample_rate / 2
        normalized_cutoff = cutoff_freq / nyquist
        b, a = signal.butter(order, normalized_cutoff, btype='high')
        return signal.filtfilt(b, a, signal)
        
    def bandpass_filter(self, signal, low_freq, high_freq, order=4):
        """バンドパスフィルタ"""
        nyquist = self.sample_rate / 2
        low_normalized = low_freq / nyquist
        high_normalized = high_freq / nyquist
        b, a = signal.butter(order, [low_normalized, high_normalized], btype='band')
        return signal.filtfilt(b, a, signal)
        
    def notch_filter(self, signal, center_freq, q_factor=30):
        """ノッチフィルタ"""
        nyquist = self.sample_rate / 2
        normalized_freq = center_freq / nyquist
        b, a = signal.iirnotch(normalized_freq, q_factor)
        return signal.filtfilt(b, a, signal)
        
    def parametric_eq(self, signal, center_freq, gain_db, q_factor=1.0):
        """パラメトリックイコライザ"""
        # 二次IIRフィルタによるピーキング
        w0 = 2 * np.pi * center_freq / self.sample_rate
        A = 10**(gain_db / 40)
        alpha = np.sin(w0) / (2 * q_factor)
        
        # フィルタ係数
        b0 = 1 + alpha * A
        b1 = -2 * np.cos(w0)
        b2 = 1 - alpha * A
        a0 = 1 + alpha / A
        a1 = -2 * np.cos(w0)
        a2 = 1 - alpha / A
        
        # 正規化
        b = np.array([b0, b1, b2]) / a0
        a = np.array([1, a1, a2]) / a0
        
        return signal.filtfilt(b, a, signal)
        
    def wah_effect(self, signal, rate=1.0, min_freq=200, max_freq=2000):
        """ワウ効果（周波数が変化するバンドパスフィルタ）"""
        t = np.arange(len(signal)) / self.sample_rate
        
        # LFOによる周波数変調
        lfo = 0.5 * (1 + np.sin(2 * np.pi * rate * t))
        center_freq = min_freq + (max_freq - min_freq) * lfo
        
        # 時間変化するフィルタ（フレームごとに処理）
        frame_size = 1024
        output = np.zeros_like(signal)
        
        for i in range(0, len(signal) - frame_size, frame_size // 2):
            frame = signal[i:i + frame_size]
            frame_center = i + frame_size // 2
            
            if frame_center < len(center_freq):
                current_freq = center_freq[frame_center]
                bandwidth = current_freq * 0.3  # 30%の帯域幅
                
                try:
                    filtered_frame = self.bandpass_filter(
                        frame, current_freq - bandwidth/2, current_freq + bandwidth/2
                    )
                    output[i:i + frame_size] += filtered_frame * 0.5
                except:
                    output[i:i + frame_size] += frame * 0.5
                    
        return output

# テスト信号（より複雑な倍音構造）
filter_fx = FilterEffects(sample_rate)

# ホワイトノイズ + トーン
np.random.seed(42)
noise = np.random.normal(0, 0.3, len(t))
tones = (np.sin(2 * np.pi * 150 * t) +
         0.7 * np.sin(2 * np.pi * 400 * t) +
         0.5 * np.sin(2 * np.pi * 800 * t) +
         0.3 * np.sin(2 * np.pi * 1200 * t))

complex_signal = noise + tones * envelope

# フィルター効果を適用
filter_effects = {
    '元の信号': complex_signal,
    'ローパス (600Hz)': filter_fx.lowpass_filter(complex_signal, 600),
    'ハイパス (300Hz)': filter_fx.highpass_filter(complex_signal, 300),
    'バンドパス (300-800Hz)': filter_fx.bandpass_filter(complex_signal, 300, 800),
    'ノッチ (400Hz)': filter_fx.notch_filter(complex_signal, 400),
    'EQ (+6dB at 800Hz)': filter_fx.parametric_eq(complex_signal, 800, 6),
}

# フィルター応答の可視化
plt.figure(figsize=(18, 15))

for i, (effect_name, filtered_signal) in enumerate(filter_effects.items()):
    # 時間波形
    plt.subplot(6, 3, i*3 + 1)
    plt.plot(t[:2000], filtered_signal[:2000], linewidth=1)
    plt.title(f'{effect_name}（波形）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # スペクトル
    plt.subplot(6, 3, i*3 + 2)
    fft_result = fft(filtered_signal)
    frequencies = fftfreq(len(filtered_signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    
    magnitude_db = 20 * np.log10(np.abs(fft_result[positive_mask]) + 1e-10)
    plt.plot(frequencies[positive_mask], magnitude_db, linewidth=1)
    plt.title(f'{effect_name}（スペクトル）')
    plt.ylabel('振幅 (dB)')
    plt.grid(True)
    plt.xlim(0, 2000)
    
    # 元信号との差分
    plt.subplot(6, 3, i*3 + 3)
    if i > 0:  # 元信号以外
        diff = filtered_signal - complex_signal
        plt.plot(t[:2000], diff[:2000], 'r-', linewidth=1)
        plt.title('差分（フィルタ効果）')
        plt.ylabel('振幅差')
        
        # RMS値を表示
        rms_diff = np.sqrt(np.mean(diff**2))
        plt.text(0.1, 0.8, f'RMS差: {rms_diff:.3f}', 
                transform=plt.gca().transAxes,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))
    else:
        plt.text(0.5, 0.5, '（参照信号）', ha='center', va='center',
                transform=plt.gca().transAxes, fontsize=14)
        plt.title('参照')
    
    plt.grid(True)
    
    if i == len(filter_effects) - 1:
        plt.xlabel('時間 (秒)')
        plt.subplot(6, 3, i*3 + 2)
        plt.xlabel('周波数 (Hz)')
        plt.subplot(6, 3, i*3 + 3)
        plt.xlabel('時間 (秒)')

plt.tight_layout()
plt.show()

# ワウ効果の特別な可視化
wah_signal = filter_fx.wah_effect(complex_signal[:len(t)//2], rate=0.5, min_freq=200, max_freq=1500)

plt.figure(figsize=(15, 8))

# ワウ効果のスペクトログラム
plt.subplot(2, 2, 1)
f, t_spec, Sxx = signal.spectrogram(wah_signal, sample_rate, nperseg=512, noverlap=256)
plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
plt.title('ワウ効果のスペクトログラム')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 2000)
plt.colorbar(label='振幅 (dB)')

# 元信号のスペクトログラム（比較用）
plt.subplot(2, 2, 2)
f_orig, t_orig, Sxx_orig = signal.spectrogram(complex_signal[:len(wah_signal)], sample_rate, nperseg=512, noverlap=256)
plt.pcolormesh(t_orig, f_orig, 10 * np.log10(Sxx_orig + 1e-10), shading='gouraud', cmap='plasma')
plt.title('元信号のスペクトログラム')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 2000)
plt.colorbar(label='振幅 (dB)')

# 時間波形比較
plt.subplot(2, 1, 2)
t_wah = np.arange(len(wah_signal)) / sample_rate
plt.plot(t_wah, complex_signal[:len(wah_signal)], 'b-', alpha=0.7, label='元信号', linewidth=1)
plt.plot(t_wah, wah_signal, 'r-', label='ワウ効果', linewidth=1)
plt.title('ワウ効果の時間波形比較')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print("フィルター効果の特徴:")
print("- ローパス: 高周波成分を減衰")
print("- ハイパス: 低周波成分を減衰")
print("- バンドパス: 特定帯域のみ通過")
print("- ノッチ: 特定周波数を除去")
print("- パラメトリックEQ: 特定周波数を強調/減衰")
print("- ワウ: 動的なバンドパスフィルタ")

## 3. 高度な音響効果

In [ ]:
class AdvancedEffects:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def pitch_shift(self, signal, shift_semitones):
        """ピッチシフト（PSOLA風の簡易実装）"""
        shift_ratio = 2**(shift_semitones / 12.0)
        
        # STFTベースのピッチシフト
        f, t, Zxx = signal.stft(signal, self.sample_rate, nperseg=1024, noverlap=512)
        
        # 周波数軸を拡大・縮小
        shifted_Zxx = np.zeros_like(Zxx)
        
        for i in range(Zxx.shape[0]):
            new_freq_idx = int(i / shift_ratio)
            if 0 <= new_freq_idx < Zxx.shape[0]:
                shifted_Zxx[i, :] = Zxx[new_freq_idx, :]
                
        # 逆STFT
        _, shifted_signal = signal.istft(shifted_Zxx, self.sample_rate, nperseg=1024, noverlap=512)
        
        return shifted_signal
        
    def ring_modulation(self, signal, modulator_freq=30):
        """リングモジュレーション"""
        t = np.arange(len(signal)) / self.sample_rate
        modulator = np.sin(2 * np.pi * modulator_freq * t)
        return signal * modulator
        
    def bit_crush(self, signal, bits=8):
        """ビットクラッシュ効果"""
        # 量子化レベル
        levels = 2**bits
        
        # 正規化
        normalized = signal / np.max(np.abs(signal))
        
        # 量子化
        quantized = np.round(normalized * (levels - 1)) / (levels - 1)
        
        return quantized * np.max(np.abs(signal))
        
    def granular_synthesis(self, signal, grain_size=1024, overlap=0.5, randomness=0.1):
        """グラニュラーシンセシス効果"""
        hop_size = int(grain_size * (1 - overlap))
        output = np.zeros(len(signal))
        
        # ハニング窓
        window = np.hanning(grain_size)
        
        for i in range(0, len(signal) - grain_size, hop_size):
            # グレインを抽出
            grain = signal[i:i + grain_size] * window
            
            # ランダムな位置オフセット
            if randomness > 0:
                offset = int(np.random.uniform(-randomness, randomness) * grain_size)
                start_pos = max(0, i + offset)
                end_pos = min(len(output), start_pos + grain_size)
                
                if end_pos > start_pos:
                    grain_len = end_pos - start_pos
                    output[start_pos:end_pos] += grain[:grain_len]
            else:
                output[i:i + grain_size] += grain
                
        return output
        
    def flanger(self, signal, rate=0.5, depth=0.005, delay=0.005):
        """フランジャー効果"""
        delay_samples = int(delay * self.sample_rate)
        max_depth_samples = int(depth * self.sample_rate)
        
        t = np.arange(len(signal)) / self.sample_rate
        lfo = np.sin(2 * np.pi * rate * t)
        
        # 可変遅延
        variable_delay = delay_samples + max_depth_samples * lfo
        
        output = signal.copy()
        
        for i in range(len(signal)):
            delay_idx = i - variable_delay[i]
            
            if delay_idx >= 0 and delay_idx < len(signal) - 1:
                idx_floor = int(delay_idx)
                idx_ceil = idx_floor + 1
                frac = delay_idx - idx_floor
                
                delayed_sample = (signal[idx_floor] * (1 - frac) + 
                                signal[idx_ceil] * frac)
                
                # フィードバック付きでミックス
                output[i] = signal[i] + delayed_sample * 0.7
                
        return output

# 高度な効果のテスト
advanced_fx = AdvancedEffects(sample_rate)

# テスト信号（より音楽的）
musical_phrase = np.zeros_like(t)
note_duration = len(t) // 4

# 4つの音符
notes = [220, 246.94, 277.18, 293.66]  # A, B, C#, D
for i, note_freq in enumerate(notes):
    start_idx = i * note_duration
    end_idx = (i + 1) * note_duration
    note_t = t[start_idx:end_idx] - t[start_idx]
    
    # 音符（倍音付き）
    note = (np.sin(2 * np.pi * note_freq * note_t) +
           0.5 * np.sin(2 * np.pi * note_freq * 2 * note_t))
    
    # エンベロープ
    note_envelope = np.exp(-note_t * 2)
    musical_phrase[start_idx:end_idx] = note * note_envelope

# 高度な効果を適用
advanced_effects = {
    '元の楽句': musical_phrase,
    'ピッチシフト (+7半音)': advanced_fx.pitch_shift(musical_phrase, 7),
    'リングモジュレーション': advanced_fx.ring_modulation(musical_phrase, 30),
    'ビットクラッシュ (4bit)': advanced_fx.bit_crush(musical_phrase, 4),
    'グラニュラー': advanced_fx.granular_synthesis(musical_phrase, 512, 0.7, 0.2),
    'フランジャー': advanced_fx.flanger(musical_phrase, 0.3, 0.003, 0.008)
}

plt.figure(figsize=(18, 15))

for i, (effect_name, processed_signal) in enumerate(advanced_effects.items()):
    # 処理信号の長さを調整
    if len(processed_signal) > len(musical_phrase):
        processed_signal = processed_signal[:len(musical_phrase)]
    elif len(processed_signal) < len(musical_phrase):
        # ゼロパディング
        padded = np.zeros(len(musical_phrase))
        padded[:len(processed_signal)] = processed_signal
        processed_signal = padded
    
    # 時間波形
    plt.subplot(6, 3, i*3 + 1)
    plt.plot(t, processed_signal, linewidth=1)
    plt.title(f'{effect_name}（波形）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # スペクトログラム
    plt.subplot(6, 3, i*3 + 2)
    f_spec, t_spec, Sxx = signal.spectrogram(processed_signal, sample_rate, 
                                           nperseg=256, noverlap=128)
    plt.pcolormesh(t_spec, f_spec, 10 * np.log10(Sxx + 1e-10), 
                  shading='gouraud', cmap='viridis')
    plt.title(f'{effect_name}（スペクトログラム）')
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 2000)
    
    # 瞬時スペクトル（中間点）
    plt.subplot(6, 3, i*3 + 3)
    mid_point = len(processed_signal) // 2
    window_size = 1024
    start_idx = max(0, mid_point - window_size // 2)
    end_idx = min(len(processed_signal), start_idx + window_size)
    
    segment = processed_signal[start_idx:end_idx]
    if len(segment) == window_size:
        fft_segment = fft(segment * np.hanning(window_size))
        freqs_segment = fftfreq(window_size, 1/sample_rate)
        positive_mask = freqs_segment >= 0
        
        magnitude_db = 20 * np.log10(np.abs(fft_segment[positive_mask]) + 1e-10)
        plt.plot(freqs_segment[positive_mask], magnitude_db, linewidth=1)
    
    plt.title(f'{effect_name}（中点スペクトル）')
    plt.ylabel('振幅 (dB)')
    plt.grid(True)
    plt.xlim(0, 2000)
    
    if i == len(advanced_effects) - 1:
        plt.xlabel('時間 (秒)')
        plt.subplot(6, 3, i*3 + 2)
        plt.xlabel('時間 (秒)')
        plt.subplot(6, 3, i*3 + 3)
        plt.xlabel('周波数 (Hz)')

plt.tight_layout()
plt.show()

print("高度な音響効果の特徴:")
print("1. ピッチシフト: 音程を変更（時間は保持）")
print("2. リングモジュレーション: 金属的な音色")
print("3. ビットクラッシュ: デジタル歪み、ローファイ効果")
print("4. グラニュラーシンセシス: 粒状の質感")
print("5. フランジャー: うねりのある効果")
print("6. これらの効果は組み合わせて使用することも可能")

## 練習問題

1. 複数の音響効果を組み合わせたエフェクトチェーンを作成してみましょう
2. リアルタイムで効果のパラメータを変更できるインターフェースを実装してみましょう
3. 楽器固有の音色をシミュレートする音響効果を設計してみましょう